In [0]:
from pyspark.sql.functions import col, regexp_replace, lit, current_timestamp

storageAccountName = "satcseguroimoveis"

def salvar_silver(df_silver, tabela): 
    df_silver.write \
        .format("delta") \
        .mode('overwrite') \
        .save(f"/mnt/{storageAccountName}/silver/{tabela}")

def renomear_colunas(df, tabela):
    for coluna in df.columns:
        col_format = coluna.upper()
        col_format = col_format.replace("ID_", "CODIGO_")
        col_format = col_format.replace("DT_", "DATA_")

        df = df.withColumnRenamed(coluna, col_format)
    df = df.drop("DATA_INSERT_BRONZE")
    df = df.drop("FILENAME")
    df = df.withColumn("FILENAME_BRONZE", lit(tabela))
    df = df.withColumn("DATA_INSERT_SILVER", current_timestamp())
    return df

def transformar_cliente():
    df_cliente = spark.read.format("delta").load(f"/mnt/{storageAccountName}/bronze/cliente")

    df_cliente = df_cliente.withColumn('telefone', regexp_replace('telefone', '[^0-9]', ''))
    df_cliente = df_cliente.withColumn('telefone', col('telefone').substr(-10, 10))
    df_cliente = df_cliente.withColumnRenamed('dt_nasc', 'data_nascimento')

    df = renomear_colunas(df_cliente, "cliente")

    salvar_silver(df, "cliente")


def transformar(tabela):
    df_bronze = spark.read.format("delta").load(f"/mnt/{storageAccountName}/bronze/{tabela}")
    
    df = renomear_colunas(df_bronze, tabela)
    
    salvar_silver(df, tabela)


In [0]:
tables = ["apolice_cobertura", "cobertura", "sinistro", "pagamento", "apolice", "avaliacao", "imovel", "cliente"]

for table in tables:
    if table == "cliente":
        transformar_cliente()
    else:
        transformar(table)

    print(f"Transformação da tabela: {table} finalizada.")

Transformação da tabela: apolice_cobertura finalizada.
Transformação da tabela: cobertura finalizada.
Transformação da tabela: sinistro finalizada.
Transformação da tabela: pagamento finalizada.
Transformação da tabela: apolice finalizada.
Transformação da tabela: avaliacao finalizada.
Transformação da tabela: imovel finalizada.
Transformação da tabela: cliente finalizada.


In [0]:
df_apolice = spark.read.format("delta").load("/mnt/satcseguroimoveis/silver/apolice_cobertura")
df_apolice.show(15)

+--------------+----------------+-----------------+--------------------+
|CODIGO_APOLICE|CODIGO_COBERTURA|  FILENAME_BRONZE|  DATA_INSERT_SILVER|
+--------------+----------------+-----------------+--------------------+
|             1|               5|apolice_cobertura|2024-06-27 03:11:...|
|             1|              10|apolice_cobertura|2024-06-27 03:11:...|
|             1|               6|apolice_cobertura|2024-06-27 03:11:...|
|             2|               1|apolice_cobertura|2024-06-27 03:11:...|
|             2|               9|apolice_cobertura|2024-06-27 03:11:...|
|             2|               3|apolice_cobertura|2024-06-27 03:11:...|
|             2|               8|apolice_cobertura|2024-06-27 03:11:...|
|             3|               5|apolice_cobertura|2024-06-27 03:11:...|
|             3|              10|apolice_cobertura|2024-06-27 03:11:...|
|             3|               1|apolice_cobertura|2024-06-27 03:11:...|
|             3|               7|apolice_cobertura|